In [ ]:
# @title YouTube Importer & Transcriber
# @markdown このセルを実行すると、動画の文字起こしとブログ記事構成案の作成を一括で行います。
# @markdown 実行後、URLの入力が求められます。

import sys
import subprocess
import os
import time
from pathlib import Path
from datetime import datetime
from typing import Dict, Tuple

# --- 1. 環境設定 & ライブラリインストール ---
def install_dependencies():
    print("環境を確認中...")
    # numpyバージョンの指定を削除
    required = ["yt-dlp>=2025.01.12", "openai-whisper==20240930"]
    need_install = False
    
    try:
        import yt_dlp
        import whisper
        import numpy
    except ImportError:
        need_install = True

    if need_install:
        print("必要なライブラリをインストールしています...（数分かかる場合があります）")
        subprocess.check_call([sys.executable, "-m", "pip", "install"] + required)
        subprocess.check_call(["apt", "install", "-y", "ffmpeg"])
        print("インストール完了。")
    else:
        print("ライブラリはインストール済みです。")

install_dependencies()

# --- 2. 必要なモジュールのインポート ---
try:
    import whisper
    from yt_dlp import YoutubeDL
except ImportError:
    import site
    site.main()
    import whisper
    from yt_dlp import YoutubeDL
except ValueError as e:
    if "numpy" in str(e):
        print("!"*60)
        print("【重要】Numpyのバージョン競合が発生しました。")
        print("メニューの「ランタイム」→「セッションを再起動」を選択し、もう一度このセルを実行してください。")
        print("!"*60)
    raise e

# --- 3. コード定義 ---

# === youtube_downloader.py ===


from pathlib import Path
from typing import Dict, Tuple

from yt_dlp import YoutubeDL


def download_audio_with_metadata(url: str, tmp_dir: Path) -> Tuple[Path, Dict]:
    """
    指定したYouTube URLから音声（または動画）をダウンロードし、
    保存ファイルパスとメタデータを返す。

    - Androidクライアント設定のみを使用します。
    - CookieファイルやPOトークンは一切使用しません。
    """
    tmp_dir.mkdir(parents=True, exist_ok=True)

    output_template = str(tmp_dir / "%(id)s.%(ext)s")

    # Androidクライアント設定のみを使用
    ydl_opts = {
        # 音声のみ（できればm4a）を取得。Whisper側でそのまま渡せる形式にする。
        "format": "bestaudio[ext=m4a]/bestaudio/best",
        "outtmpl": output_template,
        "noplaylist": True,
        # 端末ログを最小限にする
        "quiet": True,
        "no_warnings": True,
        # YouTubeのボット検出をある程度回避するための設定（Androidクライアント固定）
        "extractor_args": {
            "youtube": {
                "player_client": ["android"],
                "player_skip": ["webpage", "configs"],
            }
        },
        # User-AgentをAndroidクライアントに合わせる
        "user_agent": "com.google.android.youtube/19.09.37 (Linux; U; Android 11) gzip",
        # 追加のHTTPヘッダー
        "http_headers": {
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
            "Accept-Language": "en-us,en;q=0.5",
            "Accept-Encoding": "gzip, deflate",
            "Accept-Charset": "ISO-8859-1,utf-8;q=0.7,*;q=0.7",
            "Keep-Alive": "300",
            "Connection": "keep-alive",
        },
        # リトライ設定
        "retries": 5,
        "fragment_retries": 5,
        # タイムアウト設定
        "socket_timeout": 60,
        # 待機時間を追加（レート制限回避）
        "sleep_interval": 1,
        "max_sleep_interval": 5,
    }

    with YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)

    # 実際に保存されたファイルパスを info から取得
    audio_path: Path | None = None
    requested = info.get("requested_downloads") or []
    if requested:
        audio_path = Path(requested[0].get("filepath"))
    elif "_filename" in info:
        audio_path = Path(info["_filename"])

    if not audio_path or not audio_path.exists() or audio_path.stat().st_size == 0:
        raise FileNotFoundError("音声ファイルのダウンロードに失敗しました（ファイルが空か存在しません）。")

    metadata = {
        "title": info.get("title"),
        "uploader": info.get("uploader"),
        "duration": info.get("duration"),
        "description": info.get("description"),
    }

    return audio_path, metadata




# === whisper_runner.py ===


from pathlib import Path

import whisper


_model = None


def _get_model():
    global _model
    if _model is None:
        # モデルサイズは必要に応じて変更可能（tiny, base, small, medium, large）
        _model = whisper.load_model("small")
    return _model


def transcribe_audio(audio_path: Path) -> tuple[str, str]:
    """
    mp3音声ファイルをWhisperで文字起こししてテキストと検出言語を返す。
    
    Returns:
        tuple[str, str]: (文字起こしテキスト, 検出言語コード)
    """
    model = _get_model()
    # language を指定しないことで、Whisper に自動言語判定させる
    # これにより、英語など日本語以外の動画にも対応できる
    result = model.transcribe(str(audio_path), verbose=False)
    text = result.get("text", "").strip()
    detected_language = result.get("language", "unknown")
    return text, detected_language




# === main.py ===



from pathlib import Path
from datetime import datetime





def get_prompt_header(num_videos: int) -> str:
    """
    動画数に応じたプロンプトヘッダーを生成する。
    """
    if num_videos > 1:
        video_note = f"\n**重要: 以下の{num_videos}つの動画の文字起こしから、1つの統合されたブログ記事を作成してください。複数の動画の内容を関連付けながら、1つのまとまった記事として構成してください。**\n\n**注意: 複数の動画はそれぞれ異なる言語で文字起こしされている可能性があります（例: 1つ目の動画が日本語、2つ目の動画が英語など）。各動画の文字起こしテキストの言語を適切に理解し、最終的な記事は必ず自然な日本語で執筆してください。**\n"
    else:
        video_note = ""
    
    return f"""
## 1. ブログ記事の生成
以下の「【YouTubeの概要とその動画の文字起こし】」の内容に基づき、以下の「記事の仕様」を**全て厳守**して、SEOに強いブログ記事を生成してください。
{video_note}
なお、文字起こしの内容には誤認識や抜け漏れが含まれている可能性があります。そのため、必要に応じて内容を補完・要約しながら、自然で読みやすいブログ記事になるように構成してください。
文字起こし自体の言語が日本語以外（例: 英語）であっても、最終的な記事は必ず自然な日本語で執筆してください。

### 記事の仕様
#### 1. 構成形式

- **記事タイトル（h1）:** SEOを意識した魅力的なタイトルで、内容が一目で分かるものにする。

- **序論:**
    - 見出しなしで本文のみを記述。
    - 記事の背景、問題提起、読者の興味を引く導入文、この記事で何が分かるかを明示する。
    - **結びは必ず「ぜひ最後までご覧ください。」とする。**

- **本文（h2〜h4の階層構造）:**
    - 適切な階層構造で情報を整理する（h2 > h3 > h4）。
    - h2を大きなトピック、h3をh2の中の小トピック、h4をさらに詳細な内容とする。
    - h2よりもh3を多めに設置する。
    - 記事全体が極端に長くなりそうな場合は、本文を「前半」と「後半」に分けて生成する。
        - 最初のレスポンスでは、序論から本文の「前半」までを出力する。
        - ユーザーから「後半も続けて」などのメッセージが送られた場合に、前半の続きとして本文の「後半」とまとめを出力する。

- **まとめ（h2）:**
    - 記事全体の要約、重要なポイントの再確認、今後の展望や読者へのメッセージを含める。
    - **結びは必ず「最後までご覧いただき、ありがとうございます。」とする。**


#### 2. 文章の書き方とトーン

- 自然な日本語を用い、AI特有の機械的な表現を避ける。
- 口語的な表現（例：「〜ですね」「〜でしょう」）を適度に使用し、親しみやすいトーンで読者に語りかける。
- 適度に体言止めなどの表現も使用し、文章にリズムと変化を持たせて読みやすくする。
- 具体例や数字を積極的に用いる。
- 箇条書きは最小限にとどめ、基本的に文章で説明する。
- **h2などの文字は本文中に入れない。**
- **必ず「。」が来たら段落を変える。**
- **文字起こしや動画概要にリンク（URL）が含まれている場合は、そのリンクをブログ記事の中に適切に記述してください。リンクは文中に自然に組み込むか、関連する箇所で明示的に紹介してください。**
- **比較や一覧、数値データなど、表にまとめた方が読者にとってわかりやすい部分は、適度に表（テーブル）を使用して整理してください。表は見やすく、理解しやすい形式で作成してください。**


#### 3. 避けるべき表現

- 過度にフォーマルな表現
- 機械的な列挙
- テンプレート的な文章
- 「〜について解説します」などの定型文の多用
- 「非常に」「大変」「～の通り」「～でしょうか」など、AIが多用しがちな強調表現や定型的な口語表現


---
## 2. スラッグ提案
上記で生成した記事の内容に基づき、SEOに有利な半角英数字のスラッグ（URLの一部）を3つ提案して、記事の末尾に追記してください。
ただし、スラッグはあまり長くならないようにすること。
---



### 【YouTubeの概要とその動画の文字起こし】"""


# ここに文字起こししたいYouTubeのURLを設定してください。
# 
# 設定方法（優先順位順）:
# 1. 環境変数 YOUTUBE_URL または YOUTUBE_URLS が設定されている場合はそちらを優先します
#    - 複数のURLはカンマ区切りで指定できます
#    - 例: export YOUTUBE_URLS='https://www.youtube.com/watch?v=...,https://www.youtube.com/watch?v=...'
# 2. 環境変数がない場合、以下の YOUTUBE_URLS 変数に直接URLを設定してください
#    - 1つのURLでもリスト形式で指定してください
#    - 例: YOUTUBE_URLS = ["https://www.youtube.com/watch?v=..."]
#    - 例: YOUTUBE_URLS = ["https://www.youtube.com/watch?v=...", "https://www.youtube.com/watch?v=..."]
#
# 注意: 一部の動画は特に制限されている可能性があります。別の動画で試してください。
import os

# ここに直接YouTubeのURLを設定できます（リスト形式）
# 環境変数 YOUTUBE_URL または YOUTUBE_URLS が設定されている場合はそちらが優先されます
# 環境変数がない場合、以下の変数に直接URLを設定してください:
# 例: YOUTUBE_URLS = ["https://www.youtube.com/watch?v=..."]
# 例: YOUTUBE_URLS = ["https://www.youtube.com/watch?v=...", "https://www.youtube.com/watch?v=..."]
YOUTUBE_URLS = [
    "https://www.youtube.com/watch?v=AT7muM2RpBs"
]  # ここに直接URLを設定してください（リスト形式）


def run(urls: list[str]) -> None:
    """
    複数のYouTube URLを処理して、1つのtranscriptファイルにまとめる。
    """
    base_dir = Path(__file__).resolve().parent
    tmp_dir = base_dir / "tmp"
    tmp_dir.mkdir(exist_ok=True)

    # transcriptを最大5件まで保存（6件目以降は一番古いファイルを削除）
    transcripts_dir = base_dir / "transcripts"
    transcripts_dir.mkdir(exist_ok=True)

    existing = sorted(
        transcripts_dir.glob("transcript_*.txt"),
        key=lambda p: p.stat().st_mtime,
    )
    while len(existing) >= 5:
        oldest = existing.pop(0)
        try:
            oldest.unlink()
        except Exception:
            pass

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    out_path = transcripts_dir / f"transcript_{timestamp}.txt"

    video_data_list = []

    # 各URLを処理
    for idx, url in enumerate(urls, 1):
        print(f"[{idx}/{len(urls)}] 処理中: {url}")
        
        # 複数動画の場合、前の動画処理後に待機時間を設ける（ボット検出回避）
        if idx > 1:
            wait_time = 3  # 3秒待機
            print(f"     （ボット検出回避のため{wait_time}秒待機中...）")
            time.sleep(wait_time)
        
        try:
            audio_path, metadata = download_audio_with_metadata(url, tmp_dir)
        except Exception as e:
            error_str = str(e)
            # ボット検出エラーの場合、より分かりやすいメッセージを表示
            if "bot" in error_str.lower() or "Sign in to confirm" in error_str:
                print(f"[{idx}/{len(urls)}] ⚠️  エラー: この動画はボット検出によりダウンロードできませんでした")
                print(f"     URL: {url}")
                print(f"     対処法: 別の動画で試すか、しばらく時間をおいてから再試行してください")
            else:
                print(f"[{idx}/{len(urls)}] ⚠️  エラー: YouTubeのダウンロードに失敗しました")
                print(f"     URL: {url}")
                print(f"     詳細: {error_str[:200]}")  # 長いエラーメッセージは最初の200文字のみ
            continue
        
        title = metadata.get("title") or "(不明)"
        uploader = metadata.get("uploader") or "(不明)"
        duration = metadata.get("duration")
        description = metadata.get("description") or "(概要なし)"

        try:
            text, detected_language = transcribe_audio(audio_path)
            # 言語コードを日本語名に変換
            language_names = {
                "ja": "日本語",
                "en": "英語",
                "ko": "韓国語",
                "zh": "中国語",
                "es": "スペイン語",
                "fr": "フランス語",
                "de": "ドイツ語",
                "it": "イタリア語",
                "pt": "ポルトガル語",
                "ru": "ロシア語",
                "ar": "アラビア語",
                "hi": "ヒンディー語",
                "th": "タイ語",
                "vi": "ベトナム語",
            }
            language_display = language_names.get(detected_language, detected_language)
            print(f"[{idx}/{len(urls)}] 検出言語: {language_display} ({detected_language})")
        except Exception as e:
            import traceback
            print(f"[{idx}/{len(urls)}] ⚠️  エラー: Whisperでの文字起こしに失敗しました")
            print(f"     詳細: {str(e)[:200]}")  # 長いエラーメッセージは最初の200文字のみ
            try:
                audio_path.unlink(missing_ok=True)
            except Exception:
                pass
            continue
        finally:
            try:
                audio_path.unlink(missing_ok=True)
            except Exception:
                pass

        video_data_list.append({
            "title": title,
            "uploader": uploader,
            "url": url,
            "duration": duration,
            "description": description,
            "transcript": text,
        })
        print(f"[{idx}/{len(urls)}] ✓ 完了")

    if not video_data_list:
        print("[エラー] 処理に成功した動画がありませんでした。")
        return

    # ファイルに書き込み
    with out_path.open("w", encoding="utf-8") as f:
        # 先頭にプロンプトとガイドライン（動画数に応じて変更）
        prompt_header = get_prompt_header(len(video_data_list))
        f.write(prompt_header + "\n\n")

        # 各動画の情報を書き込み（タイトル、チャンネル、URL、長さ、概要）
        for idx, video_data in enumerate(video_data_list, 1):
            if len(video_data_list) > 1:
                f.write(f"## 動画 {idx}/{len(video_data_list)}\n\n")
            
            f.write(f"タイトル: {video_data['title']}\n")
            f.write(f"チャンネル: {video_data['uploader']}\n")
            f.write(f"URL: {video_data['url']}\n")
            if video_data['duration'] is not None:
                minutes = int(video_data['duration']) // 60
                seconds = int(video_data['duration']) % 60
                f.write(f"長さ: 約 {minutes} 分 {seconds} 秒\n")
            f.write("\n--- 動画概要 ---\n")
            f.write(f"{video_data['description']}\n")
            
            if idx < len(video_data_list):
                f.write("\n\n")

        # 文字起こしセクションを1回だけ出力
        f.write("\n--- 文字起こし ---\n\n")
        
        # 各動画の文字起こしを出力
        for idx, video_data in enumerate(video_data_list, 1):
            if len(video_data_list) > 1:
                # 複数動画の場合
                if idx == 1:
                    f.write("【1つ目の動画】\n\n")
                elif idx == 2:
                    f.write("【2つ目の動画】\n\n")
                elif idx == 3:
                    f.write("【3つ目の動画】\n\n")
                else:
                    f.write(f"【{idx}つ目の動画】\n\n")
            else:
                # 1つの動画のみの場合
                pass  # 見出しなし
            
            f.write(video_data['transcript'])
            
            if idx < len(video_data_list):
                f.write("\n\n")

    # 最終的な成功メッセージのみを表示
    print(f"\n✓ 完了: transcripts ディレクトリに {out_path.name} を保存しました。")
    print(f"   処理した動画数: {len(video_data_list)}/{len(urls)}")
    if len(video_data_list) < len(urls):
        failed_count = len(urls) - len(video_data_list)
        print(f"   ⚠️  {failed_count}つの動画でエラーが発生しました（上記のエラーメッセージを確認してください）")




# --- 4. 実行 & ダウンロード ---
print("\n" + "="*60)
print("  YouTube Importer & Transcriber (Whisper)")
print("="*60 + "\n")

# URL入力
url_input = input("YouTubeのURLを入力してください（複数ある場合はカンマ区切り）: ").strip()

if not url_input:
    print("URLが入力されませんでした。処理を終了します。")
else:
    # URLリストの作成
    target_urls = [u.strip() for u in url_input.replace("　", "").replace(" ", "").split(",") if u.strip()]
    
    if target_urls:
        print(f"\n以下の{len(target_urls)}件の動画を処理します:")
        for u in target_urls:
            print(f" - {u}")
        print("-" * 30 + "\n")
        
        # 実行
        run(target_urls)
        
        # 結果のダウンロード
        from google.colab import files as colab_files
        import glob
        
        print("\n処理が完了しました。最新のファイルを検索しています...")
        transcripts_dir = Path("transcripts")
        txt_files = sorted(transcripts_dir.glob("transcript_*.txt"), key=lambda p: p.stat().st_mtime)
        
        if txt_files:
            latest_file = txt_files[-1]
            print(f"ダウンロードを開始します: {latest_file.name}")
            colab_files.download(str(latest_file))
        else:
            print("生成されたファイルが見つかりませんでした。")
    else:
        print("有効なURLが見つかりませんでした。")
